In [8]:
from string import punctuation
from os import listdir
from nltk.corpus import stopwords
from pickle import dump
import string

In [9]:
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [10]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [11]:
# load all docs in a directory
def process_docs(directory, is_trian):
    documents = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # skip any reviews in the test set
        if is_trian and filename.startswith('cv9'):
            continue
        if not is_trian and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_doc(path)
        # clean doc
        tokens = clean_doc(doc)
        # add to list
        documents.append(tokens)
    return documents

In [12]:
# save a dataset to file
def save_dataset(dataset, filename):
    dump(dataset, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [13]:
# load all training reviews
negative_docs = process_docs('datasets/review_polarity/txt_sentoken/neg', True)
positive_docs = process_docs('datasets/review_polarity/txt_sentoken/pos', True)
trainX = negative_docs + positive_docs
trainy = [0 for _ in range(900)] + [1 for _ in range(900)]
save_dataset([trainX,trainy], 'models/imdb_train.pkl')

Saved: models/imdb_train.pkl


In [14]:
trainy[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
# load all test reviews
negative_docs = process_docs('datasets/review_polarity/txt_sentoken/neg', False)
positive_docs = process_docs('datasets/review_polarity/txt_sentoken/pos', False)
testX = negative_docs + positive_docs
testY = [0 for _ in range(100)] + [1 for _ in range(100)]
save_dataset([testX,testY], 'models/imdb_test.pkl')

Saved: models/imdb_test.pkl


In [31]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import load_model

In [17]:
# load a clean dataset
def load_dataset(filename):
    return load(open(filename, 'rb'))

In [18]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [19]:
# calculate the maximum document length
def max_length(lines):
    return max([len(s) for s in lines])

In [20]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [21]:
# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='models/multichannel.png')
    return model

In [22]:
# load training dataset
trainLines, trainLabels = load_dataset('models/imdb_train.pkl')

In [23]:
# create tokenizer
tokenizer = create_tokenizer(trainLines)

In [24]:
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)

Max document length: 1380
Vocabulary size: 44277


In [25]:
# encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)

(1800, 1380)


In [26]:
# define model
model = define_model(length, vocab_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1380)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1380, 100)    4427700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [27]:
# fit model
model.fit([trainX,trainX,trainX], array(trainLabels), epochs=10, batch_size=16)

Epoch 1/10
1800/1800 [==============================] - 43s 24ms/step - loss: 0.6941 - acc: 0.5444
Epoch 2/10
1800/1800 [==============================] - 44s 24ms/step - loss: 0.5008 - acc: 0.7667
Epoch 3/10
1800/1800 [==============================] - 42s 24ms/step - loss: 0.0899 - acc: 0.9689
Epoch 4/10
1800/1800 [==============================] - 41s 23ms/step - loss: 0.0063 - acc: 1.0000
Epoch 5/10
1800/1800 [==============================] - 41s 23ms/step - loss: 0.0019 - acc: 1.0000
Epoch 6/10
1800/1800 [==============================] - 41s 23ms/step - loss: 0.0012 - acc: 1.0000
Epoch 7/10
1800/1800 [==============================] - 42s 23ms/step - loss: 8.8242e-04 - acc: 1.0000
Epoch 8/10
1800/1800 [==============================] - 44s 24ms/step - loss: 6.5699e-04 - acc: 1.0000
Epoch 9/10
1800/1800 [==============================] - 43s 24ms/step - loss: 5.3815e-04 - acc: 1.0000
Epoch 10/10
1800/1800 [==============================] - 44s 24ms/step - loss: 4.3466e-04 - acc: 

In [28]:
# save the model
model.save('models/imdb_model.h5')

In [29]:
testLines, testLabels = load_dataset('models/imdb_test.pkl')
testX = encode_text(tokenizer, testLines, length)
print(trainX.shape, testX.shape)

(1800, 1380) (200, 1380)


In [32]:
# load the model
model = load_model('models/imdb_model.h5')

In [33]:
# evaluate model on training dataset
loss, acc = model.evaluate([trainX,trainX,trainX], array(trainLabels), verbose=0)
print('Train Accuracy: %f' % (acc*100))
 
# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX,testX,testX],array(testLabels), verbose=0)
print('Test Accuracy: %f' % (acc*100))

Train Accuracy: 100.000000
Test Accuracy: 84.000000
